In [2]:
import pickle
import pandas as pd
import numpy as np
import pickle

import copy
# data_root_dir : /ts/projects/mimic4_cxr/tabular/preprocess/processed_data/mimic4/

In [3]:
task = 'mortality'
target_days = 90

# cp -r /ts/projects/mimic4_cxr/tabular/preprocess/processed_data/mimic4/ /home/data/2025_MIMICIV_processed/
hosp_adm_dict = pickle.load(open(f"/home/data/2025_MIMICIV_processed/mimic4/hosp_adm_dict_{target_days}days.pkl", "rb"))

with open(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/SSL_tabular_ids.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
train_ids = data['train']
valid_ids = data['val']

In [5]:
train_dict = {adm_id: hosp_adm_dict[str(adm_id)] for adm_id in train_ids}
valid_dict = {adm_id: hosp_adm_dict[str(adm_id)] for adm_id in valid_ids}
test_dict = {adm_id: hosp_adm_dict[str(adm_id)] for adm_id in data['test']}
train_key_label_dict = {key : getattr(value, task) for key, value in train_dict.items()}
valid_key_label_dict = {key : getattr(value, task) for key, value in valid_dict.items()}

In [6]:
len(train_dict), len(valid_dict), len(test_dict), len(train_dict) + len(valid_dict) + len(test_dict), len(hosp_adm_dict)

(109321, 15617, 31235, 156173, 389761)

In [11]:
noisy_rate = 0.1

for train_noisy_rate in np.arange(0.0, 1.0, 0.1) :
    for valid_noisy_rate in np.arange(0.0, 1.0, 0.1) : 
        for test_noisy_rate in np.arange(0.0, 1.0, 0.1) : 
            np.random.seed(2026)
            new_label_dict = {
                'tabular' : {},
                'note' : {}
            }
            
            for modality in ['tabular', 'note'] : 
                for key, value in train_key_label_dict.items() : 
                    if np.random.rand() < train_noisy_rate:
                        new_label_dict[modality][key] = True if value == False else False
                    else : 
                        new_label_dict[modality][key] = value
                for key, value in valid_key_label_dict.items() : 
                    if np.random.rand() < valid_noisy_rate:
                        new_label_dict[modality][key] = True if value == False else False
                    else : 
                        new_label_dict[modality][key] = value
                for key, value in test_dict.items() : 
                    if np.random.rand() < test_noisy_rate:
                        new_label_dict[modality][key] = True if getattr(value, task) == False else False
                    else : 
                        new_label_dict[modality][key] = getattr(value, task)
            
            # with open(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/noisy_train_{train_noisy_rate:.1f}_valid_{valid_noisy_rate:.1f}_test_{test_noisy_rate:.1f}_key_labels.pkl", "wb") as f:
            #     pickle.dump(new_label_dict, f)
            
            train_valid_keys = list(train_key_label_dict.keys()) + list(valid_key_label_dict.keys())
            new_train_valid_labels = {key: new_label_dict['tabular'][key] for key in train_valid_keys}
            print(f"Original Label Distribution (train + valid) : {pd.Series(list(train_key_label_dict.values()) + list(valid_key_label_dict.values())).value_counts().to_dict()}")
            print(f"Noisy Ratio (train / valid / test) : {train_noisy_rate:.1f} / {valid_noisy_rate:.1f} / {test_noisy_rate:.1f}")
            for modality in ['tabular', 'note'] : 
                print(f"  {modality} Noisy Label Distribution")#  (train + valid) : {pd.Series(list(new_train_valid_labels.values())).value_counts().to_dict()}")
                print(f"  Train : {pd.Series([new_label_dict[modality][key] for key in train_key_label_dict.keys()]).value_counts().to_dict()}")
                print(f"  Valid : {pd.Series([new_label_dict[modality][key] for key in valid_key_label_dict.keys()]).value_counts().to_dict()}")
                print(f"  Test  : {pd.Series([new_label_dict[modality][key] for key in test_dict.keys()]).value_counts().to_dict()}")
            print()
            
            with open(f"/home/data/2025_MIMICIV_processed/mimic4/task:{task}_{target_days}days/noisy_train_{train_noisy_rate:.1f}_valid_{valid_noisy_rate:.1f}_test_{test_noisy_rate:.1f}_key_labels.pkl", "wb") as f:
                pickle.dump(new_label_dict, f)

Original Label Distribution (train + valid) : {False: 113467, True: 11471}
Noisy Ratio (train / valid / test) : 0.0 / 0.0 / 0.0
  tabular Noisy Label Distribution
  Train : {False: 99244, True: 10077}
  Valid : {False: 14223, True: 1394}
  Test  : {False: 28353, True: 2882}
  note Noisy Label Distribution
  Train : {False: 99244, True: 10077}
  Valid : {False: 14223, True: 1394}
  Test  : {False: 28353, True: 2882}

Original Label Distribution (train + valid) : {False: 113467, True: 11471}
Noisy Ratio (train / valid / test) : 0.0 / 0.0 / 0.1
  tabular Noisy Label Distribution
  Train : {False: 99244, True: 10077}
  Valid : {False: 14223, True: 1394}
  Test  : {False: 25793, True: 5442}
  note Noisy Label Distribution
  Train : {False: 99244, True: 10077}
  Valid : {False: 14223, True: 1394}
  Test  : {False: 25854, True: 5381}

Original Label Distribution (train + valid) : {False: 113467, True: 11471}
Noisy Ratio (train / valid / test) : 0.0 / 0.0 / 0.2
  tabular Noisy Label Distributi